# Casual Forest

Random Forestをカーネル重み付けとみなす視点に、Robinson (1988)の部分線形モデルを合わせたもの

[一般化ランダムフォレストの理論と統計的因果推論への応用 - Speaker Deck](https://speakerdeck.com/tomoshige_n/ban-hua-randamuhuoresutonoli-lun-totong-ji-de-yin-guo-tui-lun-henoying-yong?slide=42)

- [ランダムフォレストによる因果推論と最近の展開 - Speaker Deck](https://speakerdeck.com/tomoshige_n/randamuhuoresutoniyoruyin-guo-tui-lun-tozui-jin-nozhan-kai-838c3989-570d-49ca-b630-22044a798589)

## Causal Tree

### 背景：Random Forestの理論解析の進展

- Delgado et al. (2014) はUCIデータセット121に対して様々な手法を適用し、Random Forestが優れた結果を示したと報告
    - [Fernández-Delgado, M., Cernadas, E., Barro, S., & Amorim, D. (2014). Do we need hundreds of classifiers to solve real world classification problems?. The journal of machine learning research, 15(1), 3133-3181.](https://jmlr.org/papers/volume15/delgado14a/delgado14a.pdf)
    - その後はXGBoostやLightGBMも登場しており現在は違う結果となる可能性も高いが、少ない計算量・シンプルな手法ながら優れた結果ではあった
- 漸近正規性が証明される → 信頼区間が得られる
    - Wager (2014) は無限の木があるときのRandom Forestの予測値が漸近正規性をもつことを証明
        - [Wager, S. (2014). Asymptotic theory for random forests. arXiv preprint arXiv:1405.0352.](https://arxiv.org/abs/1405.0352)
    - Mentch & Hooker (2014) は有限の木での漸近正規性を証明
        - [Mentch, L., & Hooker, G. (2014). Ensemble trees and clts: Statistical inference for supervised learning. stat, 1050, 25.](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=be56bcaad2650c02017fb455808b047befc04c9e)
- Scornet (2016) はRandom Forestがカーネル法の一種であることを示した
    - [Scornet, E. (2016). Random forests and kernel methods. IEEE Transactions on Information Theory, 62(3), 1485-1500.](https://arxiv.org/abs/1502.03836)


### 因果推論についての前提

:::{card} Potential outcome framework

- $Y_{a=1}, Y_{a=0} \in \mathbb{R}$ ：潜在結果変数
- $X_j$： $p$次元の pre-treatment 共変量（ $j=1:p$ ）
- $A=\{0,1\}$ ：処置変数
- $\pi(x)=\operatorname{Pr}(A=1 \mid X=x)$ ：傾向スコア

:::


:::{card} Assumption: Consistency

$$
Y=A Y_{a=1}+(1-A) Y_{a=0}
$$

:::

:::{card} Assumption : Unconfoundedness

$$
A \perp Y_a \mid X \text { for } a=0,1
$$
:::

:::{card} Assumption : Posititvity

$$
0<\pi(x)<1
$$

:::

:::{card} Definition: Average Treatment Effect (ATE)

$$
\theta^{A T E}=\mathrm{E}\left[Y_{a=1}-Y_{a=0}\right]
$$

:::

:::{card} Definition: Heterogeneous Treatment Effect (HTE)

$$
\theta^{H T E}(x)=\mathrm{E}\left[Y_{a=1}-Y_{a=0} \mid X=x\right]
$$

:::



### Honest

- Causal Treesはrecursive partitioningを用いてHeterogeneous Treatment Effectを推定する手法。
- honest性という概念がcausal forestsやGeneralized Random Forestの証明において重要な役割を果たす。
- またhonest性を満たすTreeはCARTと比較して過学習を起こしにくいという性質もある。

:::{card} honest

**「木の分割（partitioning）をするために用いるサンプル」と「TreeのLeafごとの推定量の計算に用いるサンプル」に別々のサンプルを用いることで、partition $\Pi$ と 推定量$\hat{\mu}$ が独立になったTree を honest なTreeであるという**

:::


#### honestな木はCARTと異なる目的関数をもつ

honestな木はpartition $\Pi$のもとで estimation sample $\mathcal{S}^{e s t}$ を用いて推定された条件付き平均$\hat{\mu}\left(X_i ; \mathcal{S}^{e s t}, \Pi\right)$とテストデータ$\mathcal{S}^{t e}$の平均二乗誤差

$$
\operatorname{MSE}\left(\mathcal{S}^{t e}, \mathcal{S}^{e s t}, \Pi\right)=\frac{1}{\#\left(\mathcal{S}^{t e}\right)} 
\sum_{i \in \mathcal{S}^{t e}}
\left\{\left(Y_i-\hat{\mu}\left(X_i ; \mathcal{S}^{e s t}, \Pi\right)\right)^2-Y_i^2\right\}
$$

の期待値をとったものを最小化する。


$$
\Pi^{\text{honest}}=
\arg\min_\Pi \mathrm{E}_{\mathcal{S}^{\text{te}}, \mathcal{S}^{\text{est}}, \mathcal{S}^{\text{tr}}}
\left[\operatorname{MSE}(\mathcal{S}^{\text{te}}, \mathcal{S}^{\text{est}}, \Pi(\mathcal{S}^{\text{tr}})\right]
$$

一方で一般的なCARTでは、訓練サンプル $\mathcal{S}^{\text{te}}$ を使ってpartition $\Pi$と推定量$\hat{\mu}$を作って誤差を最小化する

$$
\Pi^{\text{CART}}=
\arg\min_\Pi \mathrm{E}_{\mathcal{S}^{\text{te}}, \mathcal{S}^{\text{tr}}}
\left[\operatorname{MSE}(\mathcal{S}^{\text{te}}, \mathcal{S}^{\text{tr}}, \Pi(\mathcal{S}^{\text{tr}})\right]
$$



#### honestな木は過学習しにくい

MSEの期待値を取ったものをEMSE

$$
\operatorname{EMSE}(\Pi) := \mathrm{E}_{\mathcal{S}^{t e}, \mathcal{S}^{\text {est }}}\left[\operatorname{MSE}\left(\mathcal{S}^{t e}, \mathcal{S}^{\text {est }}, \Pi\right)\right]
$$

とする。honestな木はこれを目的関数とする。

負のEMSEを展開すると

$$
\begin{aligned}
-\operatorname{EMSE}(\Pi) & =-\mathrm{E}_{\left(Y_i, X_j\right), \mathcal{S}^{\operatorname{est}}}\left[\left(Y_i-\mu\left(X_i ; \Pi\right)^2-Y_i\right]\right. \\
& -\mathrm{E}_{X_i, \mathcal{S}^{\text {est }}}\left[\left(\hat{\mu}\left(X_i ; \mathcal{S}^{\text {est }} ; \Pi\right)-\mu\left(X_i ; \Pi\right)\right)^2\right] \\
& =\mathrm{E}_{X_i}\left[\mu^2\left(X_i ; \Pi\right)\right]-\mathrm{E}_{\mathcal{S}^{\text {est }}, X_i}\left[\operatorname{Var}\left(\hat{\mu}\left(X_i ; \mathcal{S}^{\text {est }} ; \Pi\right)\right)\right]
\end{aligned}
$$

となる。

これに対して訓練サンプル$\mathcal{S}^{t r}$から不偏推定量を構成すると

$$
\widehat{\operatorname{EMSE}}\left(\mathcal{S}^{t r}, \Pi\right)
=\frac{1}{N^{t r}} \sum_{i \in \mathcal{S}^{t r}} \hat{\mu}^2\left(X_i ; \mathcal{S}^{t r}, \Pi\right)
-\underbrace{ \frac{2}{N^{t r}} \cdot \sum_{\ell \in \Pi} S_{\mathcal{S}^{t r}}^2(\ell) }_{penalty}
$$

となる。ここで$S_{\mathcal{S}^{t r}}^2(\ell)$は$\ell \in \Pi$におけるleaf内分散を意味する。

一方で、CARTにおいてはpenalty項がなく、分割を行えば行うほど$-\operatorname{MSE}$が改善するため、枝刈りが必要になる。

$$
-\operatorname{MSE}\left(\mathcal{S}^{t r}, \mathcal{S}^{t r}, \Pi\right)=\frac{1}{N^{t r}} \sum_{i \in \mathcal{S}^{t r}} \hat{\mu}^2\left(X_i ; \mathcal{S}^{t r}, \Pi\right)
$$


leaf内分散はleaf内のサンプル数が多いうちは小さい（=CARTとhonest treeは似た挙動になる）が、leaf内サンプルが小さくなると高くなりやすい（分割を停止する方向に動く）。


#### HTEの推定

問題：データ $\left(Y_i, X_i, W_i\right) \in \mathbb{R} \times \mathbb{R}^\rho \times\{0,1\}$ が観測されたもとで、 $\theta^{H T E}(x)=\mathrm{E}\left[Y_{a=1}-Y_{a=0} \mid X=x\right]$ を推定する問題


$$
\tau(x ; \Pi) \equiv \mathrm{E}\left[Y_{a=1}-Y_{a=0} \mid X \in \ell(x ; \Pi)\right]
$$

$$
\mu(a, x ; \Pi) \equiv \mathrm{E}\left[Y_a \mid X \in \ell(x ; \Pi)\right]
$$

## Causal Forest

## 応用研究



### 小売業においてクーポン配布のCATEを推定

[How causal machine learning can leverage marketing strategies: Assessing and improving the performance of a coupon campaign | PLOS One](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0278937)

- Kaggle公開データを使ってCATEを推定
- 配布した5種類のクーポンのうち、ドラッグストア商品券と食品カテゴリ券の2種だけが有意に売上増効果を持つことが分かった。
- 他のクーポンは統計的に効果が確認できず、無効な施策だった。
- また効果の異質性分析から、「ドラッグストア券は事前購買額の大きい優良顧客に特に効果的」「一般食品券は購買額の少ないライト層に効果的」といったセグメント別の効果差が判明した
- 例えば、既存ヘビーユーザーには日用品系クーポンで追加購買を促進でき、ライトユーザーには普段買わない食品クーポンで来店を促すと効果的。
- この結果を踏まえ、クーポン配布の対象を効果の高いセグメントに絞る最適施策を導出したところ、全顧客に配布する場合と比べ費用対効果が大幅に向上した（少数配布で売上効果の大半を実現）。因果フォレストを用いることで、マーケティングROIを最大化する配布戦略の策定が可能になることを示した。


### Amazonの顧客ターゲティング

[[2409.13847] Segment Discovery: Enhancing E-commerce Targeting](https://arxiv.org/abs/2409.13847)

方法
- Amazon社内の大規模実験データ（オンラインサービス等のABテスト複数）。例：ある定額サービスで解約防止のプロモメッセージを送る実験（数十万人規模）。
- まずランダム化比較試験から顧客毎のCATEをCausal Forestやuplift modelで推定。次に企業側の制約（予算・接触可能数）を組み込んだ最適化アルゴリズム（Large-Scale Budget-Constrained Causal Forest）で最大効果のターゲット集合を選定。

結果
- 従来は、例えば離脱率の高い順に顧客に介入するなど予測モデルに基づくターゲティングが行われていた。しかし実験データを分析したところ、離脱リスクが極端に高い顧客層では、プロモメッセージを送っても効果が無いばかりか逆効果（介入で離脱率が悪化）になる場合があることが判明
- 一方、リスクが中程度の層に大きなプラス効果が見られるなど、どの層に介入すべきかは単純な予測スコア順とは一致しないことが示された。
- Causal Forestを用いるターゲティングでは、こうした「介入効果の高い層」を的確に抽出できる。実際、**あるサービスの例では従来ルールでは離脱スコア下位39.1%を一律介入対象としていたが、因果モデルに基づき効果の見込める顧客だけを選別したところ、むしろ全体の半数以下の配信で離脱抑止効果の75%以上を達成**できた。
- このアプローチにより、顧客体験を損ねる無駄な介入を避けつつ、限られたマーケ予算で最大限の効果を上げることが可能となる。Amazon社の研究では複数のキャンペーンで有効性が検証されており、大規模パーソナライズドマーケティングへの因果機械学習の実用性を示すケースとなっている。

### 「少しずつ頻繁に」行う宿題の学習効果 McJames et al. (2024)

- [Little and often: Causal inference machine learning demonstrates the benefits of homework for improving achievement in mathematics and science - ScienceDirect](https://www.sciencedirect.com/science/article/pii/S0959475224000951?via%3Dihub)
- [Study reveals impact of homework on student achievement in math and science](https://phys.org/news/2024-09-reveals-impact-homework-student-math.html)

方法：

- 国際学力調査TIMSS 2019のデータから、アイルランドの中学2年生4,118人の数学・理科成績と家庭学習時間を分析
- Bayesian Causal Forests で、宿題の頻度・長さを疑似介入変数として成績への因果効果を推定

結果：
- 週あたりの宿題実施回数を増やすことが学力向上に効果的である一方、1回あたりの宿題時間を延ばしても効果は頭打ちになる傾向が示された
- 15分程度の短い課題でも長時間の課題と同等の効果があり、頻度を上げて宿題を小分けにする「Little and Often」戦略が有効であることがデータから裏付けられた。
- **数学では「毎日少し」の宿題が最も成績を伸ばし** 、理科では週3～4回程度が効果的。
- また従来の通説では家庭環境により宿題効果に差が出ると考えられていたが、本解析ではどの社会経済層の生徒も宿題の恩恵を等しく享受しており、一部の層だけが得をするわけではないことが示された。


## 参考文献

- [Causal TreeとCausal Forestの理論と実装 - tomtom58’s blog](https://tomtom58.hatenablog.com/entry/2024/07/15/211016)
    - フルスクラッチ実装例がある
- [計量経済学と 機械学習の交差点入り口 （公開用） | PDF](https://www.slideshare.net/slideshow/ss-67818872/67818872)